In [1]:
%reload_ext autoreload
%autoreload all

%aimport -torch
%aimport -matplotlib
%aimport -seaborn
%aimport -numpy
%aimport -pandas
%aimport -scipy
%aimport -lightning 

In [2]:
from eeg_snn_encoder.config import PROCESSED_DATA_DIR
from eeg_snn_encoder.dataset import CHBMITDataset, CHBMITDataModule

# Load the dataset
dataset = CHBMITDataset(PROCESSED_DATA_DIR / "stft_normalized.h5")

2025-05-09 07:10:07.929 | INFO     | eeg_snn_encoder.config:<module>:11 - PROJ_ROOT path is: /workspace/snn-encoder-test


In [3]:
datamodule = CHBMITDataModule(dataset, batch_size=512, worker=0)

In [5]:
from eeg_snn_encoder.tuning import filter_and_sort_trials

pe_studies = [
    {"name": "Classifier_PE_Tuning", "filter_fn": lambda t: t.value / t.params["phase_window"] < 12},
    {"name": "model-tuning-pe-mse-1", "filter_fn": lambda x: x.value < 0.24},
]

test_trials = filter_and_sort_trials(pe_studies)

Loading trials from Classifier_PE_Tuning
Found 23 qualifying trials in Classifier_PE_Tuning
Loading trials from model-tuning-pe-mse-1
Found 10 qualifying trials in model-tuning-pe-mse-1


In [6]:
from loguru import logger

import lightning.pytorch as pl
from lightning.pytorch.callbacks.early_stopping import EarlyStopping
import optuna
from optuna.integration import PyTorchLightningPruningCallback

from eeg_snn_encoder.encoders import PhaseEncoderExpand
from eeg_snn_encoder.models.classifier import ModelConfig
from eeg_snn_encoder.models.lightning import LitSeizureClassifier, OptimizerConfig
import torch

torch.set_float32_matmul_precision('medium')


def objective(trial: optuna.Trial) -> float:
    model_params: ModelConfig = {
        "threshold": trial.suggest_float("threshold", 0.01, 0.5),
        "slope": trial.suggest_float("slope", 1.0, 20.0),
        "beta": trial.suggest_float("beta", 0.1, 0.99),
        "dropout_rate1": trial.suggest_float("dropout_rate1", 0.1, 0.99),
        "dropout_rate2": trial.suggest_float("dropout_rate2", 0.1, 0.99),
    }

    optimizer_params: OptimizerConfig = {
        "lr": trial.suggest_float("lr", 1e-6, 1e-4, log=True),
        "weight_decay": trial.suggest_float("weight_decay", 1e-6, 1e-4, log=True),
        "scheduler_factor": trial.suggest_float("scheduler_factor", 0.1, 0.99),
        "scheduler_patience": trial.suggest_int("scheduler_patience", 1, 10),
    }

    encoder_params = {
        "phase_window": trial.suggest_int("phase_window", 1, 8),
    }

    spike_encoder = PhaseEncoderExpand(**encoder_params)
    
    lit_model = LitSeizureClassifier(
        model_config=model_params,
        optimizer_config=optimizer_params,
        spike_encoder=spike_encoder,
    )

    trainer = pl.Trainer(
        max_epochs=50,
        accelerator="auto",
        devices="auto",
        strategy="auto",
        enable_model_summary=False,
        enable_checkpointing=False,
        callbacks=[PyTorchLightningPruningCallback(trial, monitor="val_mse"), EarlyStopping(monitor="val_loss", mode="min", patience=5)],
        logger=False,
    )

    trainer.fit(lit_model, datamodule=datamodule)

    return trainer.callback_metrics["val_mse"].item()

In [7]:
import os

study = optuna.create_study(
    direction="minimize",
    study_name="model-fine-tuning-pe",
    storage=os.environ["OPTUNA_CONN_STRING"],
    load_if_exists=True,
    pruner=optuna.pruners.NopPruner()
)

[I 2025-05-09 07:12:15,522] A new study created in RDB with name: model-fine-tuning-pe


In [8]:
for trial in test_trials:
    study.enqueue_trial(params=trial.params)

study.optimize(objective, n_trials=len(test_trials))

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/workspace/snn-encoder-test/.venv/lib/python3.12/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:425: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=127` in the `DataLoader` to improve performance.
/workspace/snn-encoder-test/.venv/lib/python3.12/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:425: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=127` in the `DataLoader` to improve performance.


Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

[I 2025-05-09 07:51:29,841] Trial 16 finished with value: 0.45816734433174133 and parameters: {'threshold': 0.12444392748356303, 'slope': 9.535348913295238, 'beta': 0.9821120964203542, 'dropout_rate1': 0.8614677560820828, 'dropout_rate2': 0.5611079827457449, 'lr': 1.122906644507063e-05, 'weight_decay': 2.75326056410633e-05, 'scheduler_factor': 0.45064415674797087, 'scheduler_patience': 4, 'phase_window': 2}. Best is trial 8 with value: 0.18592298030853271.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

[W 2025-05-09 07:51:34,402] Trial 17 failed with parameters: {'threshold': 0.0980435735970105, 'slope': 11.378687327835207, 'beta': 0.8632803636899922, 'dropout_rate1': 0.44853730640193, 'dropout_rate2': 0.892661626626119, 'lr': 1.9739150332157962e-05, 'weight_decay': 5.117067179894617e-06, 'scheduler_factor': 0.5625858205309283, 'scheduler_patience': 6, 'phase_window': 3} because of the following error: OutOfMemoryError('CUDA out of memory. Tried to allocate 20.00 MiB. GPU 0 has a total capacity of 23.54 GiB of which 1.38 MiB is free. Process 6054 has 584.00 MiB memory in use. Including non-PyTorch memory, this process has 22.96 GiB memory in use. Of the allocated memory 21.00 GiB is allocated by PyTorch, and 1.49 GiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment

OutOfMemoryError: CUDA out of memory. Tried to allocate 20.00 MiB. GPU 0 has a total capacity of 23.54 GiB of which 1.38 MiB is free. Process 6054 has 584.00 MiB memory in use. Including non-PyTorch memory, this process has 22.96 GiB memory in use. Of the allocated memory 21.00 GiB is allocated by PyTorch, and 1.49 GiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [10]:
logger.info(f"Encoder: PE Encoding,trial, best_param: {study.best_params}")
logger.info(f"Encoder: PE Encoding,trial, best_score: {study.best_value}")

2025-05-09 07:53:56.608 | INFO     | __main__:<module>:1 - Encoder: PE Encoding,trial, best_param: {'threshold': 0.04686941733058056, 'slope': 18.601176807691544, 'beta': 0.6539485545573536, 'dropout_rate1': 0.6708858441132419, 'dropout_rate2': 0.41092616599946064, 'lr': 2.336754113027876e-05, 'weight_decay': 8.77917320478611e-06, 'scheduler_factor': 0.35299009619838584, 'scheduler_patience': 3, 'phase_window': 1}
2025-05-09 07:53:57.042 | INFO     | __main__:<module>:2 - Encoder: PE Encoding,trial, best_score: 0.18592298030853271
